# Benchmarking: Julia

In [1]:
using Random

In [2]:
Random.seed!(0)

TaskLocalRNG()

## Objective Functions

We will be using the [Levy3](http://infinity77.net/global_optimization/test_functions_nd_L.html#go_benchmark.Levy03), [Michalewicz](https://www.sfu.ca/~ssurjano/michal.html), and [Ackley](https://www.sfu.ca/~ssurjano/ackley.html) objective functions for benchmarking.

In [69]:
function levy3(x)
    n = length(x)

    y(x_i) = 1 + (x_i-1)/4

    term1 = sin(π*y(x[1]))^2
    term3 = y(x[n]-1)^2 * (1+sin(2π*y(x[n]))^2)
    
    sum = 0
    for i=1:n-1
        new = (y(x[i])-1)^2*(1+10sin(π*y(x[i])+1)^2)
        sum += new
    end

    return term1 + sum + term3
end

function michalewicz(x; m=10)
    sleep(0.01)
    return -sum(sin(v)*sin(i*v^2/π)^(2m) for
               (i,v) in enumerate(x))
end

function ackley(x; a=20, b=0.2, c=2π)
    d = length(x)
    return -a*exp(-b*sqrt(sum(x.^2)/d)) -
              exp(sum(cos.(c*xi) for xi in x)/d) + a + exp(1)
end

ackley (generic function with 4 methods)

## Genetic Algorithm (GA) Implementation

The following section defines a GA implementation using a real-valued representation. The Selection, Crossover, and Mutation types will be reused in both the baseline and parallelized versions of the algorithm.

### Population

In [47]:
function rand_population_uniform(m, a, b)
    d = length(a)
    return [a+rand(d).*(b-a) for i in 1:m]
end

rand_population_uniform (generic function with 1 method)

### Selection

In [7]:
abstract type SelectionMethod end
struct TruncationSelection <: SelectionMethod
    k # top k to keep
end
function select(t::TruncationSelection, y)
    p = sortperm(y)
    return [p[rand(1:t.k, 2)] for i in y]
end

struct TournamentSelection <: SelectionMethod
    k
end
function select(t::TournamentSelection, y)
    getparent() = begin
        p = randperm(length(y))
        p[argmin(y[p[1:t.k]])]
    end
    return [[getparent(), getparent()] for i in y] 
end

struct RouletteWheelSelection <: SelectionMethod end
function select(::RouletteWheelSelection, y)
    y = maximum(y) .- y
    cat = Categorical(normalize(y, 1))
    return [rand(cat, 2) for i in y]
end

select (generic function with 3 methods)

### Crossover

In [8]:
abstract type CrossoverMethod end
struct SinglePointCrossover <: CrossoverMethod end
function crossover(::SinglePointCrossover, a, b)
    i = rand(1:length(a))
    return vcat(a[1:i], b[i+1:end])
end

struct TwoPointCrossover <: CrossoverMethod end
function crossover(::TwoPointCrossover, a, b)
    n = length(a)
    i, j = rand(1:n, 2)
    if i > j
        (i,j) = (j,i)
    end
    return vcat(a[1:i], b[i+1:j], a[j+1:n])
end

struct UniformCrossover <: CrossoverMethod end
function crossover(::UniformCrossover, a, b)
    child = copy(a)
    for i in 1 : length(a)
        if rand() < 0.5
            child[i] = b[i]
        end
    end
    return child
end

crossover (generic function with 3 methods)

### Mutation

In [9]:
abstract type MutationMethod end
struct BitwiseMutation <: MutationMethod
    λ
end
function mutate(M::BitwiseMutation, child)
    return [rand() < M.λ ? !v : v for v in child]
end

struct GaussianMutation <: MutationMethod
    σ
end
function mutate(M::GaussianMutation, child)
    return child + randn(length(child))*M.σ
end

mutate (generic function with 2 methods)

### Algorithm

First, ensure that we have multithreading enabled. The easiest way to do this is to specify the number of threads when starting up the notebook:

`JULIA_NUM_THREADS=4 jupyter-notebook`

In [16]:
Threads.nthreads() # should be > 1

4

In [22]:
function genetic_algorithm(f, population, k_max, S, C, M; parallel=false)
    n = length(population)
    for k in 1 : k_max
        if parallel
            # Perform function evaluations in parallel
            f_pop = zeros(n)
            Threads.@threads for i=1:n
               f_pop[i] = f(population[i]) 
            end
        else
            f_pop = f.(population)
        end
        parents = select(S, f_pop)
        children = [crossover(C, population[p[1]], population[p[2]]) for p in parents]
        population .= mutate.(Ref(M), children)
    end
    population[argmin(f.(population))]
end

genetic_algorithm (generic function with 1 method)

### Benchmarking (baseline)

In [38]:
f = x -> michalewicz(x)

S = TruncationSelection(10)
C = SinglePointCrossover()
M = GaussianMutation(0.1)

k_max = 10
m = 40
population = rand_population_uniform(m, [0.0, 0.0], [4.0, 4.0])

40-element Vector{Vector{Float64}}:
 [1.4428945738932555, 3.6191622322854937]
 [3.1387476917146744, 2.047670021892199]
 [1.5411375055631122, 2.4844357857429977]
 [2.0663343190000125, 2.037003259837116]
 [0.22818803298950474, 2.1492219907554047]
 [3.2035561709351774, 0.11266125677973671]
 [2.414011460775739, 2.6898706009080584]
 [1.558782991361193, 3.2184448330752513]
 [3.0346905048523136, 1.9251774400046924]
 [2.9258788909058953, 2.9629305679389293]
 [2.652300018360916, 2.5910124955694003]
 [2.0012920092702924, 1.0632552728993252]
 [3.4472086054738362, 0.6172718041941527]
 ⋮
 [2.0152079541242722, 2.0512697854222934]
 [0.7822308497027923, 0.7461019165182101]
 [0.659748375472982, 1.0915537535141313]
 [2.75510439864937, 3.8466640125688505]
 [0.6715960312398699, 1.7404103129759845]
 [2.8274950768098064, 2.1765956775626853]
 [3.799436870946522, 3.1439367117858743]
 [2.8735214859103486, 0.9777126422009714]
 [3.8086790748615806, 3.3697838804643223]
 [2.2644492108350867, 0.4810047151197625]
 [

In [44]:
@time genetic_algorithm(f, population, k_max, S, C, M)

 44.869281 seconds (5.96 k allocations: 338.406 KiB)


2-element Vector{Float64}:
 2.175962301481834
 1.5867688530327524

In [45]:
@time genetic_algorithm(f, population, k_max, S, C, M; parallel=true)

 15.299984 seconds (27.42 k allocations: 1.606 MiB, 0.77% compilation time)


2-element Vector{Float64}:
 2.2136565467610883
 1.5539304324094187

## Simulated Annealing Implementation

In [253]:
function simulated_annealing(f, x, t, k_max)
    y = f(x)
    x_best, y_best = x, y
    for k in 1 : k_max
        x′ = x + randn(length(x))
        y′ = f(x′)
        Δy = y′ - y
        if Δy ≤ 0 || rand() < exp(-Δy/t(k))
            x, y = x′, y′
        end
        if y′ < y_best
            x_best, y_best = x′, y′
        end
    end
    return x_best, y_best
end

simulated_annealing (generic function with 2 methods)

In [324]:
x = [3.0, 3.0]

# fast annealing
T0 = 25
t(k) = T0/k

k_max = 1000

simulated_annealing(ackley, x, t, k_max)

([-0.022496545439987123, 9.009672518574263e-5], 0.07705081049613982)